In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn.modules.loss import *
from Loss.triplet import *
from session import *
from LR_Schedule.cyclical import Cyclical
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
from validation import _AccuracyMeter
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *
import util
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
%matplotlib notebook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
writer = SummaryWriter()

In [ ]:
torch.cuda.set_device(0); torch.backends.cudnn.benchmark=True; torch.cuda.is_available(); 
torch.cuda.get_device_name(0)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

In [ ]:
trainset = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)

idx = (trainset.targets!=5) & (trainset.targets!=7)

trainset.data = trainset.data[idx]
trainset.targets = trainset.targets[idx]

valset = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False)

In [ ]:
(trainset.targets == 7).sum()

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    Flatten(),
    nn.Linear(7 * 7 * 64, 1000),
    nn.Linear(1000, 64)
)

model

In [ ]:
criterion = BatchHardTripletLoss(1)
optim_fn = optim.Adam

In [ ]:
sess = Session(model, criterion, optim_fn, 1e-3)

In [ ]:
lr_find(sess, trainloader, start_lr=1e-7)

In [ ]:
sess.set_lr(1e-4)

In [ ]:
schedule = TrainingSchedule(trainloader, [LossLogger(), CosAnneal(len(trainloader), T_mult=2)])

In [ ]:
sess.train(schedule, 7)

In [ ]:
with EvalModel(model):
    outputs = []
    for input, label in valloader:
        outputs.append(model.forward(Variable(util.to_gpu(input))).data.cpu())
     

In [ ]:
preds = torch.cat(outputs)
preds.shape

In [ ]:
writer.add_embedding(preds, metadata=valset.targets, label_img=1.0 - valset.data.reshape(-1, 1, 28, 28) / 255.0)

In [ ]:
pca = PCA(n_components=3)
reduced = pca.fit_transform(preds)

In [ ]:
dir(valset)
valset.targets

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(reduced[:,0], reduced[:,1], reduced[:,2], c=valset.targets)